In [1]:
# Model without synthetic data
# Specify arguments

# 439
PROB_DICT = {1:1,
            3:2,
            5:3,
            12:4,
            13:5,
            232:6,
            233:7,
            234:8,
            235:9,
            236:10
           }

# # 487
# PROB_DICT = {17:1,
#             20:2,
#             21:3,
#             22:4,
#             24:5,
#             25:6,
#             28:7,
#             100:8,
#             101:9,
#             102:10
#            }

# # 492
# PROB_DICT = {31:1,
#             32:2,
#             33:3,
#             34:4,
#             36:5,
#             37:6,
#             38:7,
#             39:8,
#             40:9,
#             128:10
#            }

# # 494
# PROB_DICT = {41:1,
#             43:2,
#             44:3,
#             46:4,
#             49:5,
#             67:6,
#             104:7,
#             106:8,
#             107:9,
#             108:10
#            }

# # 502
# PROB_DICT = {45:1,
#             48:2,
#             51:3,
#             56:4,
#             57:5,
#             64:6,
#             70:7,
#             71:8,
#             112:9,
#             118:10
#            }

asmt = 439

# lrs = [0.0003, 0.0004]
# max_epochs_list = [40, 60]
# sampling_times = 2
# folds = 3

lr, max_epochs = 0.01, 120
sampling_times = 5
folds = 4

track_lrs = []
track_epochs = []
track_aucs = []
        

batch_size = 64 


import pandas as pd
kc_df = pd.read_csv("../data/CWOSyntheticNewProbSolProblem.csv")

tracking_df = kc_df
tracking_df = tracking_df.fillna(0).set_index('ProblemID')
tracking_df = tracking_df.drop(['ProblemDec', 'Hussle in ChatGPT'], axis=1)
dropping = []
for c in tracking_df.columns:
    column = tracking_df[c]
    if sum(column)/len(column)<0.25 or sum(column)/len(column)>0.75:
        dropping.append(c)


tracking_df = tracking_df.drop(dropping, axis=1)

rand_seed = 0
code_df = pd.read_csv("../data/paths_with_functions.tsv", sep='\t')

code_df = code_df[code_df['AssignmentID'] == asmt] 

correct_df = code_df[code_df['Score'] == 1]

problems = pd.unique(code_df['ProblemID'])
# synthetic_problems = [p for p in problems if p > 1000]
# original_problems = [p for p in problems if p < 1000]

from sklearn.model_selection import train_test_split, KFold


import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import random
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import sem
from matplotlib import pyplot as plt

def setup_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.random.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

setup_seed(rand_seed)

def create_word_index_table(vocab):
    """
    Creating word to index table
    Input:
    vocab: list. The list of the node vocabulary

    """
    ixtoword = {}
    # period at the end of the sentence. make first dimension be end token
    ixtoword[0] = 'END'
    ixtoword[1] = 'UNK'
    wordtoix = {}
    wordtoix['END'] = 0
    wordtoix['UNK'] = 1
    ix = 2
    for w in vocab:
        wordtoix[w] = ix
        ixtoword[ix] = w
        ix += 1
    return wordtoix, ixtoword

def convert_to_idx(sample, node_word_index, path_word_index):
    """
    Converting to the index 
    Input:
    sample: list. One single training sample, which is a code, represented as a list of neighborhoods.
    node_word_index: dict. The node to word index dictionary.
    path_word_index: dict. The path to word index dictionary.

    """
    sample_index = []
    for line in sample:
        components = line.split(",")
        if components[0] in node_word_index:
            starting_node = node_word_index[components[0]]
        else:
            starting_node = node_word_index['UNK']
        if components[1] in path_word_index:
            path = path_word_index[components[1]]
        else:
            path = path_word_index['UNK']
        if components[2] in node_word_index:
            ending_node = node_word_index[components[2]]
        else:
            ending_node = node_word_index['UNK']

        sample_index.append([starting_node,path,ending_node])
    return sample_index


# Pre-processing of the sequential data
# Cheat removal
all_students = pd.unique(code_df['SubjectID'])
code_df['ServerTimestamp'] = pd.to_datetime(code_df['ServerTimestamp'], format="%Y-%m-%dT%H:%M:%S")
def sec_diff(late, early):
    return (late - early)/ pd.Timedelta(seconds=1)


processed_df_list = []
for s in all_students:
    if pd.isna(s):
        continue
    student_df = code_df[code_df['SubjectID'] == s].reset_index()
    past_row = student_df.iloc[0]
    start_row = student_df.iloc[0]
    dropping_index = []
    cheat_flag = 0
    problem_count = 1
    current_problem_attempts = 1
    attempt_average = 0
    for i, row in student_df.iterrows():
        # Init removal time
        remove_flag = 0

        # Calculate different in submission time
        second_diff = sec_diff(row['ServerTimestamp'], past_row['ServerTimestamp'])

        # Cheat handling
        # New problem
        if row['ProblemID'] != past_row['ProblemID'] and i > 0:
            # Cheat checker (simple version)

            if (current_problem_attempts < 3 and # Not struggled
                attempt_average - current_problem_attempts > 8 and # Historical struggle
                problem_count > 3):

                time_used = sec_diff(past_row['ServerTimestamp'], start_row['ServerTimestamp'])
                if time_used < 90: # Short and fast submission
                    cheat_flag = 1
                    break

            # Update to new problem
            attempt_average = (attempt_average*problem_count + current_problem_attempts)/(problem_count + 1)
            problem_count += 1
            current_problem_attempts = 1
            start_row = row
        # Old problem
        else:
            current_problem_attempts += 1


#         # Confusion handling
#         if i > 0:
#             if row['Code'] == past_row['Code']:
#                 remove_flag = 1
#             if second_diff < 15 and row['Score'] != 1:
#                 remove_flag = 1

        # Partial handling (really simple)
        if len(row['Code'].split("\n")) <= 5:
            remove_flag = 1

        # Update row
        past_row = row

        if remove_flag == 1:
            dropping_index.append(i)
    student_df = student_df.drop(dropping_index)
    if not cheat_flag:
        processed_df_list.append(student_df)
#     processed_df_list.append(student_df)
processed_df = pd.concat(processed_df_list)

import javalang
def program_parser(func):
    tokens = javalang.tokenizer.tokenize(func)
    parser = javalang.parser.Parser(tokens)
    tree = parser.parse_member_declaration()
    return tree

# Keep first
first_df_list = []
students = pd.unique(processed_df["SubjectID"])
for s in students:
    student_df = processed_df[processed_df['SubjectID'] == s].reset_index()
    past_ps = []
    dropping_index = []
    for i, row in student_df.iterrows():
        remove_flag = 0

        # ONLY KEEP FIRST ATTEMPT!
        if row['ProblemID'] not in past_ps:
            try:
                parsed = program_parser(row['Code'])
                past_ps.append(row['ProblemID'])
            except:
                parsed = "Uncompilable"
                remove_flag = 1
        else:
            remove_flag = 1


        # Update row
        past_row = row

        if remove_flag == 1:
            dropping_index.append(i)
    student_df = student_df.drop(dropping_index)
    first_df_list.append(student_df)
first_df = pd.concat(first_df_list)

# processed_df['First'] = processed_df['CodeStateID'].isin(first_df['CodeStateID'])

processed_df = first_df # Only keep the first attempts


def read_seq_df(df, subjects, PROB_DICT):
    """
    Reading extracted path data frame and return the paths and the label.

    """
#     separated_paths_label = []
    separated_paths_data = []
    separated_paths_problem = []
    separated_paths_score = []
    for s in subjects:
        student_df = df[df['SubjectID'] == s]
#         s_separated_paths_label = []
        s_separated_paths_data = []
        s_separated_paths_problem = []
        s_separated_paths_score = []
        for index, row in student_df.iterrows():
            if type(row['RawASTPath']) == str:
#                 s_separated_paths_label.append(row[tracking_df.columns[KC_index]])
                s_separated_paths_data.append(row['RawASTPath'].split("\U0001f972"))
                index_prob_id = PROB_DICT[row['ProblemID']]
                s_separated_paths_problem.append(index_prob_id)
                s_separated_paths_score.append(row['Score'])

#         separated_paths_label.append(s_separated_paths_label)
        separated_paths_data.append(s_separated_paths_data)
        separated_paths_problem.append(s_separated_paths_problem)
        separated_paths_score.append(s_separated_paths_score)


    return separated_paths_data, separated_paths_problem, separated_paths_score


def read_seq_df_test(df, subjects, PROB_DICT):
    """
    Reading extracted path data frame and return the paths and the label.

    """
    separated_paths_label = []
    separated_paths_data = []
    separated_paths_problem = []
    separated_paths_score = []
    separated_paths_subjects = []
    for s in subjects:
        student_df = df[df['SubjectID'] == s]
        s_separated_paths_label = []
        s_separated_paths_data = []
        s_separated_paths_problem = []
        s_separated_paths_score = []
        for index, row in student_df.iterrows():
            if type(row['RawASTPath']) == str:
                s_separated_paths_label.append(row[tracking_df.columns[:3]])
                s_separated_paths_data.append(row['RawASTPath'].split("\U0001f972"))
                index_prob_id = PROB_DICT[row['ProblemID']]
                s_separated_paths_problem.append(index_prob_id)
                s_separated_paths_score.append(row['Score'])

#         
        if len(s_separated_paths_data) > 0:
            separated_paths_label.append(s_separated_paths_label)
            separated_paths_data.append(s_separated_paths_data)
            separated_paths_problem.append(s_separated_paths_problem)
            separated_paths_score.append(s_separated_paths_score)
            separated_paths_subjects.append(s)


    return separated_paths_data, separated_paths_problem, separated_paths_score, separated_paths_subjects, separated_paths_label


class CodeDKT(nn.Module):
    """
    Defining the network. 

    """
    def __init__(self, node_count, path_count, P_number, seq_dim, tracking_df, device):
        super(CodeDKT, self).__init__()
        KC_number = len(tracking_df.columns)

        self.KC_filter = torch.FloatTensor(tracking_df[tracking_df.index<1000].to_numpy()).to(device)
        self.seq_dim = seq_dim
        
        self.embed_nodes = nn.Embedding(node_count+2, 100) # adding unk and end
        self.embed_paths = nn.Embedding(path_count+2, 100) # adding unk and end
        self.embed_dropout = nn.Dropout(0.2)
        self.path_transformation_layer = nn.Linear(seq_dim+300,seq_dim+300)
        self.attention_layer = nn.Linear(seq_dim+300,1)

        self.attention_softmax = nn.Softmax(dim=2)

#         self.rnn = nn.LSTM(2*seq_dim+300, 100, batch_first=True) 
        self.rnn1 = nn.LSTM(seq_dim, 100, batch_first=True) # Without code
        self.rnn2 = nn.LSTM(seq_dim+300, 100, batch_first=True)

        self.fc = nn.Linear(200, KC_number) # With code
#         self.fc = nn.Linear(100, KC_number) # Without code
        self.KC_pred = nn.Linear(KC_number, 10)
        self.leakyReLU = nn.LeakyReLU()
        self.ReLU = nn.ReLU()
        self.sig = nn.Sigmoid()
        self.device = device
    def forward(self, starting_node_index, ending_node_index, path_index, correctness_sequence, evaluating=False):
        '''
        Parameters:
        batch: b
        codes: c (non-unified)
        node_embeddings: ne
        path_embeddings: pe
        
        '''
        
        rnn_first_part = correctness_sequence # (b,l,2q)

        rnn_attention_part = torch.stack([rnn_first_part.float()]*200,dim=-2) # (b,l,c,2q)
        starting_node_embed = self.embed_nodes(starting_node_index) # (b,l,c,1) -> (b,l,c,ne)
        ending_node_embed = self.embed_nodes(ending_node_index) # (b,l,c,1) -> (b,l,c,ne)
        path_embed = self.embed_paths(path_index) # (b,l,c,1) -> (b,l,c,pe)
        full_embed = torch.cat((starting_node_embed, ending_node_embed, path_embed, rnn_attention_part), dim=3) # (b,l,c,2ne+pe+q)
        if not evaluating:
            full_embed = self.embed_dropout(full_embed) # (b,l,c,2ne+pe+2q)
        full_embed_transformed = torch.tanh(self.path_transformation_layer(full_embed)) # (b,l,c,2ne+pe+2q)
        context_weights = self.attention_layer(full_embed_transformed) # (b,l,c,1)
        attention_weights = self.attention_softmax(context_weights) # (b,l,c,1)
        code_vectors = torch.sum(torch.mul(full_embed,attention_weights),dim=2) # (b,l,2ne+pe+2q)
        
        rnn_input = torch.cat((rnn_first_part.float(),code_vectors), dim=2)
        
#         out, hn = self.rnn(rnn_input) 
        out1, hn1 = self.rnn1(rnn_first_part.float())
        out2, hn2 = self.rnn2(code_vectors)
    
        KC_res = self.sig(self.fc(torch.cat((out1,out2), dim=2))) # With code
#         KC_res = self.sig(self.fc(out1)) # Without code
        
        # Keeping only positive weights for KCs (now is a special softmax)
        self.KC_pred.weight = nn.Parameter(self.sig(self.KC_pred.weight.data))
        self.KC_pred.weight = nn.Parameter(torch.mul(self.KC_pred.weight, self.KC_filter))
        self.KC_pred.weight = nn.Parameter(nn.functional.normalize(self.KC_pred.weight.data, dim=0))
        
        res = self.sig(self.KC_pred(KC_res)) 
        
        return res, KC_res

    def get_minibatches_idx(self, n, minibatch_size, shuffle=False, droplast=False):
        """
        Getting the minibatches given the training set

        """
        idx_list = np.arange(len(n))

        if shuffle:
            np.random.shuffle(idx_list)

        minibatches = []
        minibatch_start = 0
        if droplast:
            for i in range(math.floor(len(n) // minibatch_size)):
                minibatches.append(idx_list[minibatch_start : min(minibatch_start + minibatch_size,len(n))])
                minibatch_start += minibatch_size
        else:
            for i in range(math.ceil(len(n) / minibatch_size)):
                minibatches.append(idx_list[minibatch_start : min(minibatch_start + minibatch_size,len(n))])
                minibatch_start += minibatch_size
        return zip(range(len(minibatches)), minibatches)

#     def padding(self, index_list, max_length):
#         """
#         Padding the paths and the code.

#         """
#         padder_index = 0

#         padded_list = []
#         for sample in index_list:

#             if max_length > len(sample):
#                 # padding the paths, or just get the first "max_gram_length" of paths
#                 padding_length = max_length - len(sample)
#                 padded_list.append(sample + [padder_index]*padding_length)
#             else:
#                 padded_list.append(sample[:max_length])

#         return padded_list

#     def padding_sequence(self, student_list, num_questions):
#         max_length = 100
#         padder_index = [0]*max_length
#         padded_list = []
#         for index_list in student_list:

#             if num_questions > len(index_list):
#                 # padding the paths, or just get the first "max_gram_length" of paths
#                 padding_length = num_questions - len(index_list)
#                 padded_list.append(self.padding(index_list, max_length) + [padder_index]*padding_length)
#             else:
#                 padded_list.append(self.padding(index_list[:num_questions], max_length))

#         return padded_list

    def save_model(self, save_path):
        torch.save(self.state_dict(), save_path)

    def load_model(self, load_path):
        self.load_state_dict(torch.load(load_path))

def zeros2end(x):
    zeros_mask = x == 0
    out = torch.stack(
        [torch.cat([x[_, :][~zeros_mask[_, :]], x[_, :][zeros_mask[_, :]]], dim=0) for _ in range(x.size()[0])])
    return out

class lossFunc(nn.Module):
    def __init__(self, num_of_questions, bs, device):
        super(lossFunc, self).__init__()
        self.crossEntropy = nn.BCELoss()
        self.MSE = nn.MSELoss()
        self.num_of_questions = num_of_questions
        self.batch_size = bs
        self.device = device



    def forward(self, pred, correctness, problems, device="cuda:0"):

        binary_preds = torch.tensor([])
        ground_truths = torch.tensor([])

        list_preds = []
        for s_ind in range(len(pred)):
            padded_len = len(torch.nonzero(problems[s_ind]==0))
            extra = padded_len

#             print(s_ind, padded_len)
            for j in range(49-extra): # We cannot count the final submission in.
                current_pred = pred[s_ind,j].to('cpu')

                next_correctness = (((correctness[s_ind,j+1,:10] - correctness[s_ind,j+1,10:])+1)//2).to('cpu')
                problem_attempted_index = problems[s_ind,j+1]-1

#                 print("-----")
#                 print("next correct vec:", next_correctness)
#                 print("prob index:", problem_attempted_index)
#                 print("next correct:", next_correctness[problem_attempted_index])
#                 print("pred:", current_pred[problem_attempted_index])

#                 print("-----")
                binary_preds = torch.cat([binary_preds, torch.unsqueeze(current_pred[problem_attempted_index], 0)]) 
                ground_truths = torch.cat([ground_truths, torch.unsqueeze(next_correctness[problem_attempted_index],0)])
        loss = self.crossEntropy(binary_preds.double(), ground_truths.double())
        return loss, binary_preds, ground_truths

#     def forward_ori(self, pred, correctness, problems, device="cuda:0"):


#         binary_preds = torch.tensor([]).to(device)
#         ground_truths = torch.tensor([]).to(device)

#         for s_ind in range(len(pred)):

#             padded_len = len(torch.nonzero(problems[s_ind]==0))
#             extra = padded_len

#             delta = correctness[s_ind][:, 0:10] + correctness[s_ind][:, 10:20]  # shape: [length, questions]
#             temp = pred[s_ind][:49].mm(delta[1:].t().float())
#             index = torch.tensor([[i for i in range(49)]],dtype=torch.long).to(device)

#             p = temp.gather(0, index)[0]
#             a = (((correctness[s_ind][:, 0:10] - correctness[s_ind][:, 10:20]).sum(1) + 1) // 2)[1:]

#             p = p[:49-extra]
#             a = a[:49-extra]

#             binary_preds = torch.cat([binary_preds, p])
#             ground_truths = torch.cat([ground_truths, a])

#         loss = self.crossEntropy(binary_preds, ground_truths)
#         return loss, binary_preds, ground_truths

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

label1_df = pd.read_csv("../data/439_yang_labeling_data_1.csv")
label2_df = pd.read_csv("../data/439_yang_labeling_data_2.csv")

label_df = pd.concat([label1_df, label2_df])

problems = pd.unique(processed_df['ProblemID'])
synthetic_problems = [p for p in problems if p > 1000]
original_problems = [p for p in problems if p < 1000]

subjects = pd.unique(processed_df['SubjectID'])


test_subjects = pd.unique(label_df['SubjectID'])
train_val_subjects = pd.unique(processed_df[~processed_df['SubjectID'].isin(test_subjects)]['SubjectID'])

original_processed_df = processed_df[processed_df['ProblemID'].isin(original_problems)]
original_processed_df = original_processed_df.sort_values('ServerTimestamp')
test_df = original_processed_df[original_processed_df['SubjectID'].isin(test_subjects)]
test_df = test_df[test_df['CodeStateID'].isin(label_df['CodeStateID'])]

for c in tracking_df.columns:
    test_df[c] = [1-label_df[label_df['CodeStateID'] == x[1]['CodeStateID']][c].iloc[0] for x in test_df.iterrows()]
    
    
# synth_df = pd.DataFrame(columns=test_df.columns)
    
# for s in test_subjects:
#     subject_df = test_df[test_df['SubjectID']==s]
#     non_subject_df = test_df[test_df['SubjectID']!=s]
#     for i in range(len(subject_df)):
#         row = subject_df.iloc[i]
#         if row['Score'] == 1:
#             to_replace_p = row['ProblemID']

#             KCs_related = tracking_df.loc[to_replace_p].tolist()
#             KCs_index = [n for n, e in enumerate(KCs_related) if e == 1]
#             for k in KCs_index:
#                 target_df = non_subject_df[non_subject_df['ProblemID'] == to_replace_p]
#                 target_df = target_df[target_df[tracking_df.columns.to_list()[k]] == 0]
#                 for k_other in KCs_index:
#                     if k_other != k:
#                         target_df = target_df[target_df[tracking_df.columns.to_list()[k_other]] == 1]
#                 if len(target_df) == 0:
#                     continue
#                 replaced_row = target_df.sample().copy()

                
#                 for j, row_j in subject_df.iloc[:i].iterrows():
#                     new_row_j = row_j.copy()
#                     new_row_j['SubjectID'] = s + "_seq" + str(i) + "_kc" + str(k)
#                     synth_df = synth_df.append(new_row_j)
                
#                 replaced_row['ServerTimestamp'] = "2024-01-01 20:00:00" # Assign a super late time to ensure last one
#                 replaced_row['SubjectID'] = s + "_seq" + str(i) + "_kc" + str(k)
#                 synth_df = synth_df.append(replaced_row)

# synth_subjects = pd.unique(synth_df['SubjectID'])

# # Keeping only 75 pairs per kc.
# kc0_subjects = [e for e  in synth_subjects if e[-1] == '0'][:75]
# kc1_subjects = [e for e in synth_subjects if e[-1] == '1'][:75]
# kc2_subjects = [e for e in synth_subjects if e[-1] == '2'][:75]

# synth_df = synth_df[synth_df['SubjectID'].isin(kc0_subjects+kc1_subjects+kc2_subjects)]


test_f1 = []
test_auc = []
test_acc = []
test_precision = []
test_recall = []
# synth_interp_values = []
# test_interp_values = []
# synth_interp_values_by_kc = []
# test_interp_values_by_kc = []
# interp_acc = []
test_KC_vectors = []
test_gt_vectors = []
# synth_KC_vectors = []
# KC_evaled = []


practiced_problems = list(PROB_DICT.keys())



for i in range(sampling_times):
    
    
#     original_processed_df = processed_df[processed_df['ProblemID'].isin(original_problems)]
    processed_df = processed_df.sort_values('ServerTimestamp')
    
    train_df = processed_df[processed_df['SubjectID'].isin(train_val_subjects)]
    
#     for c in tracking_df.columns:
#         train_df[c] = [tracking_df.loc[x[1]['ProblemID'], c] for x in train_df.iterrows()]
#         val_df[c] = [tracking_df.loc[x[1]['ProblemID'], c] for x in val_df.iterrows()] # Readin labels

    train_data, train_problem, train_score = read_seq_df(train_df, train_val_subjects, PROB_DICT)
    test_data, test_problem, test_score, test_subjects_new, test_kc_label = read_seq_df_test(test_df, test_subjects, PROB_DICT)
#     synth_data, synth_problem, synth_score, synth_subjects_new = read_seq_df_test(synth_df, synth_subjects, PROB_DICT)


    node_hist = {}
    path_hist = {}
    for student_paths in train_data:
        for paths in student_paths:
            starting_nodes = [p.split(",")[0] for p in paths]
            path = [p.split(",")[1] for p in paths]
            ending_nodes = [p.split(",")[2] for p in paths]
            nodes = starting_nodes + ending_nodes
            for n in nodes:
                if not n in node_hist:
                    node_hist[n] = 1
                else:
                    node_hist[n] += 1
            for p in path:
                if not p in path_hist:
                    path_hist[p] = 1
                else:
                    path_hist[p] += 1

    node_count = len(node_hist)
    path_count = len(path_hist)

    # small frequency then abandon, for node and path
    valid_node = [node for node, count in node_hist.items()]
    valid_path = [path for path, count in path_hist.items()]

    # create ixtoword and wordtoix lists
    node_word_index, node_index_word = create_word_index_table(valid_node)
    path_word_index, path_index_word = create_word_index_table(valid_path)
    
    train_converted_data = [[convert_to_idx(sample, node_word_index, path_word_index) for sample in student_sample] for student_sample in train_data]
    test_converted_data = [[convert_to_idx(sample, node_word_index, path_word_index) for sample in student_sample] for student_sample  in test_data]
#     synth_converted_data = [[convert_to_idx(sample, node_word_index, path_word_index) for sample in student_sample] for student_sample  in synth_data]
    
    
    
    train_full_info = np.zeros(shape=[len(train_val_subjects), 50, 620]) # 
    test_full_info = np.zeros(shape=[len(test_subjects_new), 50, 620])
#     synth_full_info = np.zeros(shape=[len(synth_subjects_new), 50, 620])
    
    for s_ind in range(len(train_val_subjects)):
        current_s_attempts = len(train_score[s_ind])
        
        if current_s_attempts > 50:
            train_score[s_ind] = train_score[s_ind][:50]
            train_problem[s_ind] = train_problem[s_ind][:50]
            train_converted_data[s_ind] = train_converted_data[s_ind][:50]

#         extra = 50 - min(current_s_attempts, 50)
        for j in range(min(current_s_attempts, 50)):
            
            p_count = len(train_converted_data[s_ind][j])

            if p_count > 200:
                train_full_info[s_ind, j, :200] = [p[0] for p in train_converted_data[s_ind][j][:200]] # Starting
                train_full_info[s_ind, j, 200:400] = [p[2] for p in train_converted_data[s_ind][j][:200]] # Ending
                train_full_info[s_ind, j, 400:600] = [p[1] for p in train_converted_data[s_ind][j][:200]] # Path
            else:
                train_full_info[s_ind, j, :p_count] = [p[0] for p in train_converted_data[s_ind][j]] # Starting
                train_full_info[s_ind, j, 200:200+p_count] = [p[2] for p in train_converted_data[s_ind][j]] # Ending
                train_full_info[s_ind, j, 400:400+p_count] = [p[1] for p in train_converted_data[s_ind][j]] # Path
                
            if train_score[s_ind][j] == 1:
                # 0 index should be for the first problem, however used for padding in model.
                train_full_info[s_ind, j, 600+train_problem[s_ind][j]-1] = 1 
            else:
                train_full_info[s_ind, j, 600+train_problem[s_ind][j]+9] = 1
    
    for s_ind in range(len(test_subjects_new)):
        current_s_attempts = len(test_score[s_ind])
        
        if current_s_attempts > 50:
            test_score[s_ind] = test_score[s_ind][:50]
            test_problem[s_ind] = test_problem[s_ind][:50]
            test_converted_data[s_ind] = test_converted_data[s_ind][:50]
        
#         extra = 50 - min(current_s_attempts, 50)
        for j in range(min(current_s_attempts, 50)):
        
            p_count = len(test_converted_data[s_ind][j])
        
            if p_count > 200:
                test_full_info[s_ind, j, :200] = [p[0] for p in test_converted_data[s_ind][j][:200]] # Starting
                test_full_info[s_ind, j, 200:400] = [p[2] for p in test_converted_data[s_ind][j][:200]] # Ending
                test_full_info[s_ind, j, 400:600] = [p[1] for p in test_converted_data[s_ind][j][:200]] # Path
            else:
                test_full_info[s_ind, j, :p_count] = [p[0] for p in test_converted_data[s_ind][j]] # Starting
                test_full_info[s_ind, j, 200:200+p_count] = [p[2] for p in test_converted_data[s_ind][j]] # Ending
                test_full_info[s_ind, j, 400:400+p_count] = [p[1] for p in test_converted_data[s_ind][j]] # Path
            
            if test_score[s_ind][j] == 1:
                test_full_info[s_ind, j, 600+test_problem[s_ind][j]-1] = 1
            else:
                test_full_info[s_ind, j, 600+test_problem[s_ind][j]+9] = 1
                
                
#     for s_ind in range(len(synth_subjects_new)):
#         current_s_attempts = len(synth_score[s_ind])
        
#         if current_s_attempts > 50:
#             synth_score[s_ind] = synth_score[s_ind][:50]
#             synth_problem[s_ind] = synth_problem[s_ind][:50]
#             synth_converted_data[s_ind] = synth_converted_data[s_ind][:50]
        
# #         extra = 50 - min(current_s_attempts, 50)
#         for j in range(min(current_s_attempts, 50)):
        
#             p_count = len(synth_converted_data[s_ind][j])
        
#             if p_count > 200:
#                 synth_full_info[s_ind, j, :200] = [p[0] for p in synth_converted_data[s_ind][j][:200]] # Starting
#                 synth_full_info[s_ind, j, 200:400] = [p[2] for p in synth_converted_data[s_ind][j][:200]] # Ending
#                 synth_full_info[s_ind, j, 400:600] = [p[1] for p in synth_converted_data[s_ind][j][:200]] # Path
#             else:
#                 synth_full_info[s_ind, j, :p_count] = [p[0] for p in synth_converted_data[s_ind][j]] # Starting
#                 synth_full_info[s_ind, j, 200:200+p_count] = [p[2] for p in synth_converted_data[s_ind][j]] # Ending
#                 synth_full_info[s_ind, j, 400:400+p_count] = [p[1] for p in synth_converted_data[s_ind][j]] # Path
            
#             if synth_score[s_ind][j] == 1:
#                 synth_full_info[s_ind, j, 600+synth_problem[s_ind][j]-1] = 1
#             else:
#                 synth_full_info[s_ind, j, 600+synth_problem[s_ind][j]+9] = 1
    
#     val_converted_data += train_converted_data
    
    P_number = len(problems)
    
    model = CodeDKT(node_count, path_count, P_number, 20, tracking_df, device).to(device)
    
    loss_func = lossFunc(10, batch_size, device).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(max_epochs):
        model.train()
        
        mini_batches = model.get_minibatches_idx(train_converted_data, batch_size, shuffle=True, droplast=False)
        epoch_loss = 0
        for _, batch_index in mini_batches:
            
#             batch_samples = [train_converted_data[t] for t in batch_index]
#             raw_starting_node_index = [[[p[0] for p in single_sample] for single_sample in student_samples] for student_samples in batch_samples]
#             raw_ending_node_index = [[[p[2] for p in single_sample] for single_sample in student_samples] for student_samples in batch_samples]
#             raw_path_index = [[[p[1] for p in single_sample] for single_sample in student_samples] for student_samples in batch_samples]
#             batch_max_len = 50
#             # Padding to max paths in a file
#             starting_node_index = torch.LongTensor(model.padding_sequence(raw_starting_node_index, num_questions=batch_max_len)).to(device)
#             ending_node_index = torch.LongTensor(model.padding_sequence(raw_ending_node_index, num_questions=batch_max_len)).to(device)
#             path_index = torch.LongTensor(model.padding_sequence(raw_path_index, num_questions=batch_max_len)).to(device)
            
            batch_scores = [train_score[t] for t in batch_index]
        
            batch_problem = np.zeros(shape=[len(batch_index), 50])
            for ind, t in enumerate(batch_index):
                if len(train_problem[t]) > 50:
                    batch_problem[ind,:] = train_problem[t][:50]
                else:
                    batch_problem[ind,:len(train_problem[t])] = train_problem[t]
                    
            batch_problem = torch.LongTensor(batch_problem)
            
            batch_correctness = torch.nn.utils.rnn.pad_sequence([torch.tensor(train_full_info[t,:,600:]) for t in batch_index], batch_first=True, padding_value=0).to(device)

            starting_node_index = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(train_full_info[t,:,:200]) for t in batch_index], batch_first=True, padding_value=0).to(device)
            ending_node_index = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(train_full_info[t,:,200:400]) for t in batch_index], batch_first=True, padding_value=0).to(device)
            path_index = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(train_full_info[t,:,400:600]) for t in batch_index], batch_first=True, padding_value=0).to(device)
            
            predictions, code_vectors = model(starting_node_index, ending_node_index, path_index, batch_correctness)

            loss, binary_preds, ground_truths = loss_func(predictions, batch_correctness, batch_problem)
            
            epoch_loss += loss.item()*len(batch_index)
            # Model optimization

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        epoch_loss = epoch_loss / len(train_converted_data)
        
    #test
    model.eval()
    test_mini_batches = model.get_minibatches_idx(test_converted_data, min(batch_size,len(test_converted_data)), shuffle=False)
    test_epoch_loss = 0
    test_epoch_lc_loss = 0
    
    test_binary_preds = torch.tensor([])
    test_ground_truths = torch.tensor([])
    test_code_vectors = []
    test_raw_preds = []
    for _, test_batch_index in test_mini_batches:

        batch_scores = [test_score[t] for t in test_batch_index]

        batch_problem = np.zeros(shape=[len(test_batch_index), 50])
        for ind, t in enumerate(test_batch_index):
            if len(test_problem[t]) > 50:
                batch_problem[ind,:] = test_problem[t][:50]
            else:
                batch_problem[ind,:len(test_problem[t])] = test_problem[t]

        batch_problem = torch.LongTensor(batch_problem)

        batch_correctness = torch.nn.utils.rnn.pad_sequence([torch.tensor(test_full_info[t,:,600:]) for t in test_batch_index], batch_first=True, padding_value=0).to(device)

        starting_node_index = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(test_full_info[t,:,:200]) for t in test_batch_index], batch_first=True, padding_value=0).to(device)
        ending_node_index = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(test_full_info[t,:,200:400]) for t in test_batch_index], batch_first=True, padding_value=0).to(device)
        path_index = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(test_full_info[t,:,400:600]) for t in test_batch_index], batch_first=True, padding_value=0).to(device)
        with torch.no_grad():
            predictions, code_vectors = model(starting_node_index, ending_node_index, path_index, batch_correctness, evaluating=True)

            loss, binary_preds, ground_truths = loss_func(predictions, batch_correctness, batch_problem)

#             print(binary_preds, ground_truths)
        test_binary_preds = torch.cat([test_binary_preds, binary_preds]) 
        test_ground_truths = torch.cat([test_ground_truths, ground_truths])
        test_epoch_loss += loss.item()*len(test_batch_index)
        test_code_vectors.extend(code_vectors)
        test_raw_preds.extend(predictions)

    # Calculating the prediction scores and the classification results.
    test_loss = test_epoch_loss / len(test_converted_data)

    test_auc.append(metrics.roc_auc_score(test_ground_truths.cpu().detach().numpy(), (test_binary_preds).cpu().detach().numpy()))
    test_acc.append(metrics.accuracy_score(test_ground_truths.cpu().detach().numpy(), (test_binary_preds).cpu().detach().numpy().round())) 
    test_f1.append(metrics.f1_score(test_ground_truths.cpu().detach().numpy(), (test_binary_preds).cpu().detach().numpy().round()))
    test_precision.append(metrics.precision_score(test_ground_truths.cpu().detach().numpy(), (test_binary_preds).cpu().detach().numpy().round()))
    test_recall.append(metrics.recall_score(test_ground_truths.cpu().detach().numpy(), (test_binary_preds).cpu().detach().numpy().round()))
    
#     #synth
#     synth_mini_batches = model.get_minibatches_idx(synth_converted_data, min(batch_size,len(synth_converted_data)), shuffle=False)
#     synth_epoch_loss = 0
#     synth_epoch_lc_loss = 0
    
#     synth_binary_preds = torch.tensor([])
#     synth_ground_truths = torch.tensor([])
#     synth_code_vectors = []
#     synth_raw_preds = []
#     for _, synth_batch_index in synth_mini_batches:

#         batch_scores = [synth_score[t] for t in synth_batch_index]

#         batch_problem = np.zeros(shape=[len(synth_batch_index), 50])
#         for ind, t in enumerate(synth_batch_index):
#             if len(synth_problem[t]) > 50:
#                 batch_problem[ind,:] = synth_problem[t][:50]
#             else:
#                 batch_problem[ind,:len(synth_problem[t])] = synth_problem[t]

#         batch_problem = torch.LongTensor(batch_problem)

#         batch_correctness = torch.nn.utils.rnn.pad_sequence([torch.tensor(synth_full_info[t,:,600:]) for t in synth_batch_index], batch_first=True, padding_value=0).to(device)

#         starting_node_index = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(synth_full_info[t,:,:200]) for t in synth_batch_index], batch_first=True, padding_value=0).to(device)
#         ending_node_index = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(synth_full_info[t,:,200:400]) for t in synth_batch_index], batch_first=True, padding_value=0).to(device)
#         path_index = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(synth_full_info[t,:,400:600]) for t in synth_batch_index], batch_first=True, padding_value=0).to(device)
#         with torch.no_grad():
#             predictions, code_vectors = model(starting_node_index, ending_node_index, path_index, batch_correctness, evaluating=True)

#             loss, binary_preds, ground_truths = loss_func(predictions, batch_correctness, batch_problem)

# #             print(binary_preds, ground_truths)
#         synth_binary_preds = torch.cat([synth_binary_preds, binary_preds]) 
#         synth_ground_truths = torch.cat([synth_ground_truths, ground_truths])
# #         synth_epoch_loss += loss.item()*len(synth_batch_index)
#         synth_code_vectors.extend(code_vectors)
#         synth_raw_preds.extend(predictions)
    # Check: Synth traces (subject ID) should be matching with number of preds and truths
#     test_KC_vals = []
#     synth_KC_vals = []
    test_KC_vec = []
    test_gt_vec = []
#     target_KCs = []
#     synth_interp_values_by_kc_dict = {0:[],1:[],2:[]}
#     test_interp_values_by_kc_dict = {0:[],1:[],2:[]}
#     diff_to_probs = []
    
    for s_ind in range(len(test_subjects_new)):
#         split_subject = synth_subjects_new[s_ind].split("_")
#         trace_subject = split_subject[0]
#         seq_num = int(split_subject[1][-1])
#         kc_ind = int(split_subject[2][-1])
#         test_ind = list(test_subjects_new).index(trace_subject)
#         print(trace_subject, seq_num, kc_ind, len(synth_code_vectors))
        
        attempt_N = len(list(test_df[test_df['SubjectID'] == test_subjects_new[s_ind]]['ProblemID']))

        test_KC_vec.append([test_code_vectors[s_ind].tolist()[att] for att in range(attempt_N)])
        test_gt_vec.append([test_kc_label[s_ind][att] for att in range(attempt_N)])
        
    test_KC_vectors.append(test_KC_vec)
    test_gt_vectors.append(test_gt_vec)
#         print(test_code_vectors[test_ind])
#         print(synth_code_vectors[s_ind])
        
#         test_KC_vals.append(test_KC_val)
#         synth_KC_vals.append(synth_KC_val)
#         test_interp_values_by_kc_dict[kc_ind].append(test_KC_val)
#         synth_interp_values_by_kc_dict[kc_ind].append(synth_KC_val)
#         target_KCs.append(kc_ind)
        
#         test_KC_vec.append(test_code_vectors[test_ind][seq_num].tolist())
#         synth_KC_vec.append(synth_code_vectors[s_ind][seq_num].tolist())
        
#     test_KC_vectors.append(test_KC_vec)
#     test_gt_vectors.append(test_gt_vec)
# #     synth_interp_values.append(synth_KC_vals)
# #     test_interp_values.append(test_KC_vals)
    
#     test_interp_values_by_kc.append(test_interp_values_by_kc_dict)
#     synth_interp_values_by_kc.append(synth_interp_values_by_kc_dict)
#     KC_evaled.append(target_KCs)
    
    
#     for v_ind in range(len(test_KC_vals)):
#         diff_to_probs.append((test_KC_vals[v_ind] - synth_KC_vals[v_ind])>0)
        
#     interp_acc.append(sum(diff_to_probs)/len(diff_to_probs)) 
#     print(interp_acc)
    print("Run done:", i)

# print("avg interp acc:", np.mean(interp_acc), "std interp acc:", np.std(interp_acc), "stderr interp acc:", sem(interp_acc))

print("avg auc:", np.mean(test_auc), "std auc:", np.std(test_auc), "stderr auc:", sem(test_auc))
print("avg acc:", np.mean(test_acc), "std acc:", np.std(test_acc), "stderr acc:", sem(test_acc))
print("avg f1:", np.mean(test_f1), "std f1:", np.std(test_f1), "stderr f1:", sem(test_f1))
print("avg precision:", np.mean(test_precision), "std precision:", np.std(test_precision), "stderr precision:", sem(test_precision))
print("avg recall:", np.mean(test_recall), "std recall:", np.std(test_recall), "stderr recall:", sem(test_recall))            



# kc_val_diffs_evaled = []
# kc_val_diffs_unevaled = []

# kc_val_pos_portion_evaled = []
# kc_val_pos_portion_unevaled = []

# for f in range(sampling_times):
#     test_KC_vecs = test_KC_vectors[f]
#     synth_KC_vecs = synth_KC_vectors[f]
#     target_KCs = KC_evaled[f]
#     diff_evaled = []
#     diff_unevaled = []
#     pos_evaled = []
#     pos_unevaled = []
#     for v_ind in range(len(target_KCs)):
#         for k in range(3):
#             if k == target_KCs[v_ind]:
#                 diff_evaled.append(test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])
#                 pos_evaled.append((test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])>0)
#             else:
#                 diff_unevaled.append(test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])
#                 pos_unevaled.append((test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])>0)
#     kc_val_diffs_evaled.append(np.mean(diff_evaled))
#     kc_val_diffs_unevaled.append(np.mean(diff_unevaled))
#     kc_val_pos_portion_evaled.append(sum(pos_evaled)/len(pos_evaled))
#     kc_val_pos_portion_unevaled.append(sum(pos_unevaled)/len(pos_unevaled))
    
# print("np.mean(kc_val_diffs_unevaled):",np.mean(kc_val_diffs_unevaled))
# print("np.mean(kc_val_pos_portion_unevaled):",np.mean(kc_val_pos_portion_unevaled))

# problems_attempted = []
# for s_ind in range(len(synth_subjects_new)):
#     split_subject = synth_subjects_new[s_ind].split("_")
#     problems_attempted.append(list(synth_df[synth_df['SubjectID'] == synth_subjects_new[s_ind]]['ProblemID'])[-1])
    
# kc_val_diffs_evaled = []
# kc_val_diffs_unevaled = []
# kc_val_diffs_unrelated = []

# kc_val_pos_portion_evaled = []
# kc_val_pos_portion_unevaled = []
# kc_val_pos_portion_unrelated = []

# kc_val_diffs_evaled_split = {0:[], 1:[], 2:[]}
# kc_val_diffs_unevaled_split = {0:[], 1:[], 2:[]}
# kc_val_diffs_unrelated_split = {0:[], 1:[], 2:[]}

# kc_val_pos_portion_evaled_split = {0:[], 1:[], 2:[]}
# kc_val_pos_portion_unevaled_split = {0:[], 1:[], 2:[]}
# kc_val_pos_portion_unrelated_split = {0:[], 1:[], 2:[]}

# for f in range(sampling_times):
#     test_KC_vecs = test_KC_vectors[f]
#     synth_KC_vecs = synth_KC_vectors[f]
#     target_KCs = KC_evaled[f]
    
#     problem_changed = problems_attempted
    
#     diff_evaled = []
#     diff_unevaled = []
#     diff_unrelated = []
#     pos_evaled = []
#     pos_unevaled = []
#     pos_unrelated = []
    
#     diff_evaled_kc = {0:[], 1:[], 2:[]}
#     diff_unevaled_kc = {0:[], 1:[], 2:[]}
#     diff_unrelated_kc = {0:[], 1:[], 2:[]}
#     pos_evaled_kc = {0:[], 1:[], 2:[]}
#     pos_unevaled_kc = {0:[], 1:[], 2:[]}
#     pos_unrelated_kc = {0:[], 1:[], 2:[]}
    
#     for v_ind in range(len(target_KCs)):
#         for k in range(3):
#             if k == target_KCs[v_ind]:
#                 diff_evaled.append(test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])
#                 pos_evaled.append((test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])>0)
#                 diff_evaled_kc[k].append(test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])
#                 pos_evaled_kc[k].append((test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])>0)
#             elif k in [i for i, e in enumerate(tracking_df.loc[problems_attempted[v_ind]].tolist()) if e == 1]:
#                 diff_unevaled.append(test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])
#                 pos_unevaled.append((test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])>0)
#                 diff_unevaled_kc[k].append(test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])
#                 pos_unevaled_kc[k].append((test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])>0)
#             else:
#                 diff_unrelated.append(test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])
#                 pos_unrelated.append((test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])>0)
#                 diff_unrelated_kc[k].append(test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])
#                 pos_unrelated_kc[k].append((test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])>0)
#     kc_val_diffs_evaled.append(np.mean(diff_evaled))
#     kc_val_diffs_unevaled.append(np.mean(diff_unevaled))
#     kc_val_diffs_unrelated.append(np.mean(diff_unrelated))
#     kc_val_pos_portion_evaled.append(sum(pos_evaled)/len(pos_evaled))
#     kc_val_pos_portion_unevaled.append(sum(pos_unevaled)/len(pos_unevaled))
#     kc_val_pos_portion_unrelated.append(sum(pos_unrelated)/len(pos_unrelated))
    
#     for k in range(3):
#         kc_val_diffs_evaled_split[k].append(np.mean(diff_evaled_kc[k]))
#         kc_val_diffs_unevaled_split[k].append(np.mean(diff_unevaled_kc[k]))
#         kc_val_diffs_unrelated_split[k].append(np.mean(diff_unrelated_kc[k]))
#         kc_val_pos_portion_evaled_split[k].append(sum(pos_evaled_kc[k])/len(pos_evaled_kc[k]))
#         kc_val_pos_portion_unevaled_split[k].append(sum(pos_unevaled_kc[k])/len(pos_unevaled_kc[k]))
#         kc_val_pos_portion_unrelated_split[k].append(sum(pos_unrelated_kc[k])/len(pos_unrelated_kc[k]))
    
    
# print("np.mean(kc_val_diffs_evaled):",np.mean(kc_val_diffs_evaled))   
# print("np.mean(kc_val_diffs_unevaled):",np.mean(kc_val_diffs_unevaled))
# print("np.mean(kc_val_diffs_unrelated):",np.mean(kc_val_diffs_unrelated))
# print("np.mean(kc_val_pos_portion_evaled):",np.mean(kc_val_pos_portion_evaled))   
# print("np.mean(kc_val_pos_portion_unevaled):",np.mean(kc_val_pos_portion_unevaled))
# print("np.mean(kc_val_pos_portion_unrelated):",np.mean(kc_val_pos_portion_unrelated))

# for k in range(3):
#     print("KC:", k)
#     print("np.mean(kc_val_diffs_evaled):",np.mean(kc_val_diffs_evaled_split[k]))   
#     print("np.mean(kc_val_diffs_unevaled):",np.mean(kc_val_diffs_unevaled_split[k]))
#     print("np.mean(kc_val_diffs_unrelated):",np.mean(kc_val_diffs_unrelated_split[k]))
#     print("np.mean(kc_val_pos_portion_evaled):",np.mean(kc_val_pos_portion_evaled_split[k]))   
#     print("np.mean(kc_val_pos_portion_unevaled):",np.mean(kc_val_pos_portion_unevaled_split[k]))
#     print("np.mean(kc_val_pos_portion_unrelated):",np.mean(kc_val_pos_portion_unrelated_split[k]))


/home/yang/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:531: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629416375/work/torch/csrc/utils/python_arg_parser.cpp:766.)


Run done: 0
Run done: 1
Run done: 2
Run done: 3
Run done: 4
avg auc: 0.6956763698630137 std auc: 0.01598334109515766 stderr auc: 0.00799167054757883
avg acc: 0.6705596107055961 std acc: 0.0203334135433345 stderr acc: 0.01016670677166725
avg f1: 0.6751671427667183 std f1: 0.02855690256192617 stderr f1: 0.014278451280963084
avg precision: 0.7098040256939567 std precision: 0.01025428706095233 stderr precision: 0.005127143530476165
avg recall: 0.6447488584474885 std recall: 0.04430871039528112 stderr recall: 0.02215435519764056


In [2]:
kc_val_related = []
kc_val_unrelated = []
kc_gt_related = []
kc_gt_unrelated = []

kc_val_related_split = {0:[], 1:[], 2:[]}
kc_gt_related_split = {0:[], 1:[], 2:[]}

Q_matrix = tracking_df[tracking_df.index<1000].to_numpy()
problems_attempted = []
for s_ind in range(len(test_subjects_new)):
    problems_attempted.append(list(test_df[test_df['SubjectID'] == test_subjects_new[s_ind]]['ProblemID']))

for f in range(sampling_times):
    test_KC_vecs = test_KC_vectors[f]
    test_gt_vecs = test_gt_vectors[f]
    
    problem_changed = problems_attempted
    
    related = []
    unrelated = []
    
    related_kc = {0:[], 1:[], 2:[]}
    unrelated_kc = {0:[], 1:[], 2:[]}
    
    related_gt = []
    unrelated_gt = []
    
    related_gt_kc = {0:[], 1:[], 2:[]}
    unrelated_gt_kc = {0:[], 1:[], 2:[]}
    
    for s_ind in range(len(test_KC_vecs)):
        for v_ind in range(len(test_KC_vecs[s_ind])):
            for k in range(3):
                if k in [i for i, e in enumerate(tracking_df.loc[problems_attempted[s_ind][v_ind]].tolist()) if e == 1]:
                    test_KC_num = test_KC_vecs[s_ind][v_ind][k]
                    test_gt_num = test_gt_vecs[s_ind][v_ind][k]
                    
                    related_gt.append(test_gt_num)
                    related.append(test_KC_num)

                    related_gt_kc[k].append(test_gt_num)
                    related_kc[k].append(test_KC_num)
                    
                    if len([i for i, e in enumerate(tracking_df.loc[problems_attempted[s_ind][v_ind]].tolist()) if e == 1]) == 1:
                        related_gt.append(test_gt_num)
                        related.append(test_KC_num)

                        related_gt_kc[k].append(test_gt_num)
                        related_kc[k].append(test_KC_num)
                    
                if k in [i for i, e in enumerate(tracking_df.loc[problems_attempted[s_ind][v_ind]].tolist()) if e == 1]:
                    if len([i for i, e in enumerate(tracking_df.loc[problems_attempted[s_ind][v_ind]].tolist()) if e == 1]) == 2:
                        if test_score[s_ind][v_ind] != 1:
                            test_KC_num = test_KC_vecs[s_ind][v_ind][k]
                            test_gt_num = test_gt_vecs[s_ind][v_ind][k]

                            unrelated_gt.append(test_gt_num)
                            unrelated.append(test_KC_num)

                            unrelated_gt_kc[k].append(test_gt_num)
                            unrelated_kc[k].append(test_KC_num)
    kc_val_related.append(related)
    kc_val_unrelated.append(unrelated)
    kc_gt_related.append(related_gt)
    kc_gt_unrelated.append(unrelated_gt)
    
    
    for k in range(3):
        kc_val_related_split[k].append(related_kc[k])
        kc_gt_related_split[k].append(related_gt_kc[k])

auc_related = [metrics.roc_auc_score(kc_gt_related[f], kc_val_related[f]) for f in range(sampling_times)]
auc_unrelated = [metrics.roc_auc_score(kc_gt_unrelated[f], kc_val_unrelated[f]) for f in range(sampling_times)]
print(np.mean(auc_related), np.std(auc_related), sem(auc_related))
print(np.mean(auc_unrelated), np.std(auc_unrelated), sem(auc_unrelated))
#     kc_val_diffs_evaled.append(np.mean(diff_evaled))
#     kc_val_diffs_unevaled.append(np.mean(diff_unevaled))
#     kc_val_diffs_unrelated.append(np.mean(diff_unrelated))
#     kc_val_pos_portion_evaled.append(sum(pos_evaled)/len(pos_evaled))
#     kc_val_pos_portion_unevaled.append(sum(pos_unevaled)/len(pos_unevaled))
#     kc_val_pos_portion_unrelated.append(sum(pos_unrelated)/len(pos_unrelated))
    
#     for k in range(3):
#         kc_val_diffs_evaled_split[k].append(np.mean(diff_evaled_kc[k]))
#         kc_val_diffs_unevaled_split[k].append(np.mean(diff_unevaled_kc[k]))
#         kc_val_diffs_unrelated_split[k].append(np.mean(diff_unrelated_kc[k]))
#         kc_val_pos_portion_evaled_split[k].append(sum(pos_evaled_kc[k])/len(pos_evaled_kc[k]))
#         kc_val_pos_portion_unevaled_split[k].append(sum(pos_unevaled_kc[k])/len(pos_unevaled_kc[k]))
#         kc_val_pos_portion_unrelated_split[k].append(sum(pos_unrelated_kc[k])/len(pos_unrelated_kc[k]))
    
for k in range(3):
    print("KC:", k)
    auc_related = [metrics.roc_auc_score(kc_gt_related_split[k][f], kc_val_related_split[k][f]) for f in range(sampling_times)]
    
    print(np.mean(auc_related), np.std(auc_related), sem(auc_related))
    
# print("np.mean(kc_val_diffs_evaled):",np.mean(kc_val_diffs_evaled))   
# print("np.mean(kc_val_diffs_unevaled):",np.mean(kc_val_diffs_unevaled))
# print("np.mean(kc_val_diffs_unrelated):",np.mean(kc_val_diffs_unrelated))
# print("np.mean(kc_val_pos_portion_evaled):",np.mean(kc_val_pos_portion_evaled))   
# print("np.mean(kc_val_pos_portion_unevaled):",np.mean(kc_val_pos_portion_unevaled))
# print("np.mean(kc_val_pos_portion_unrelated):",np.mean(kc_val_pos_portion_unrelated))

# for k in range(3):
#     print("KC:", k)
#     print("np.mean(kc_val_diffs_evaled):",np.mean(kc_val_diffs_evaled_split[k]))   
#     print("np.mean(kc_val_diffs_unevaled):",np.mean(kc_val_diffs_unevaled_split[k]))
#     print("np.mean(kc_val_diffs_unrelated):",np.mean(kc_val_diffs_unrelated_split[k]))
#     print("np.mean(kc_val_pos_portion_evaled):",np.mean(kc_val_pos_portion_evaled_split[k]))   
#     print("np.mean(kc_val_pos_portion_unevaled):",np.mean(kc_val_pos_portion_unevaled_split[k]))
#     print("np.mean(kc_val_pos_portion_unrelated):",np.mean(kc_val_pos_portion_unrelated_split[k]))


NameError: name 'tracking_df' is not defined

In [3]:
np.save("CodeKCDKT_kcval.npy",[kc_val_related, kc_val_unrelated, kc_gt_related, kc_gt_unrelated])

/home/yang/anaconda3/envs/py36/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [4]:
# kc_val_diffs_evaled_save = {0:[], 1:[], 2:[]}
# kc_val_diffs_unevaled_save = {0:[], 1:[], 2:[]}
# kc_val_diffs_unrelated_save = {0:[], 1:[], 2:[]}


# for f in range(sampling_times):
#     test_KC_vecs = test_KC_vectors[f]
#     synth_KC_vecs = synth_KC_vectors[f]
#     target_KCs = KC_evaled[f]
    
#     problem_changed = problems_attempted
    
#     diff_evaled = []
#     diff_unevaled = []
#     diff_unrelated = []
#     pos_evaled = []
#     pos_unevaled = []
#     pos_unrelated = []
    
#     diff_evaled_kc = {0:[], 1:[], 2:[]}
#     diff_unevaled_kc = {0:[], 1:[], 2:[]}
#     diff_unrelated_kc = {0:[], 1:[], 2:[]}
#     pos_evaled_kc = {0:[], 1:[], 2:[]}
#     pos_unevaled_kc = {0:[], 1:[], 2:[]}
#     pos_unrelated_kc = {0:[], 1:[], 2:[]}
    
#     for v_ind in range(len(target_KCs)):
#         for k in range(3):
#             if k == target_KCs[v_ind]:
#                 diff_evaled.append(test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])
#                 pos_evaled.append((test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])>0)
#                 diff_evaled_kc[k].append(test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])
#                 pos_evaled_kc[k].append((test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])>0)
#             elif k in [i for i, e in enumerate(tracking_df.loc[problems_attempted[v_ind]].tolist()) if e == 1]:
#                 diff_unevaled.append(test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])
#                 pos_unevaled.append((test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])>0)
#                 diff_unevaled_kc[k].append(test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])
#                 pos_unevaled_kc[k].append((test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])>0)
#             else:
#                 diff_unrelated.append(test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])
#                 pos_unrelated.append((test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])>0)
#                 diff_unrelated_kc[k].append(test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])
#                 pos_unrelated_kc[k].append((test_KC_vecs[v_ind][k] - synth_KC_vecs[v_ind][k])>0)
#     kc_val_diffs_evaled.append(np.mean(diff_evaled))
#     kc_val_diffs_unevaled.append(np.mean(diff_unevaled))
#     kc_val_diffs_unrelated.append(np.mean(diff_unrelated))
#     kc_val_pos_portion_evaled.append(sum(pos_evaled)/len(pos_evaled))
#     kc_val_pos_portion_unevaled.append(sum(pos_unevaled)/len(pos_unevaled))
#     kc_val_pos_portion_unrelated.append(sum(pos_unrelated)/len(pos_unrelated))
    
#     for k in range(3):
#         kc_val_diffs_evaled_save[k].append(diff_evaled_kc[k])
#         kc_val_diffs_unevaled_save[k].append(diff_unevaled_kc[k])
#         kc_val_diffs_unrelated_save[k].append(diff_unrelated_kc[k])

In [5]:
# np.save("CodeKCDKT_interp.npy",[kc_val_diffs_evaled_save, kc_val_diffs_unevaled_save, kc_val_diffs_unrelated_save])

In [6]:
# np.save("KCDKT_usecase.npy", [test_KC_vec, test_subjects_new])

In [7]:
# len(test_KC_vec)

In [8]:
[i for i, e in enumerate(tracking_df.loc[problems_attempted[s_ind][v_ind]].tolist()) if e == 1]

[1]

In [1]:
count_1kc = 0
count_2kc = 0
count_1kc_c = 0
count_1kc_i = 0
count_2kc_c = 0
count_2kc_i = 0
for s_ind in range(len(test_KC_vecs)):
    for v_ind in range(len(test_KC_vecs[s_ind])):
        if len([i for i, e in enumerate(tracking_df.loc[problems_attempted[s_ind][v_ind]].tolist()) if e == 1]) == 2:
            count_2kc+=1
            if test_score[s_ind][v_ind] != 1:
                count_2kc_i+=1
            else:
                count_2kc_c+=1
        else:
            count_1kc+=1
            if test_score[s_ind][v_ind] != 1:
                count_1kc_i+=1
            else:
                count_1kc_c+=1

NameError: name 'test_KC_vecs' is not defined